In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [113]:
pop = 'density_variation/high'

x99 = np.load('%s/0.99/infected.npy' %pop)
x95 = np.load('%s/0.95/infected.npy' %pop)
x9 = np.load('%s/0.9/infected.npy' %pop)
x8 = np.load('%s/0.8/infected.npy' %pop)
x7 = np.load('%s/0.7/infected.npy' %pop)
x6 = np.load('%s/0.6/infected.npy' %pop)
x5 = np.load('%s/0.5/infected.npy' %pop)
x25 = np.load('%s/0.25/infected.npy' %pop)
x0 = np.load('%s/0/infected.npy' %pop)

d99 = np.load('%s/0.99/fatalities.npy' %pop)
d95 = np.load('%s/0.95/fatalities.npy' %pop)
d9 = np.load('%s/0.9/fatalities.npy' %pop)
d8 = np.load('%s/0.8/fatalities.npy' %pop)
d7 = np.load('%s/0.7/fatalities.npy' %pop)
d6 = np.load('%s/0.6/fatalities.npy' %pop)
d5 = np.load('%s/0.5/fatalities.npy' %pop)
d25 = np.load('%s/0.25/fatalities.npy' %pop)
d0 = np.load('%s/0/fatalities.npy' %pop)

In [115]:
%matplotlib qt

plt.figure(figsize=(12,8))

plt.subplot(211)
plt.plot(x99, label='99%, max cases: ' + '%i' %np.max(x95))
plt.plot(x95, label='95%, max cases: ' + '%i' %np.max(x95))
plt.plot(x9, label='90%, max cases: ' + '%i' %np.max(x9))
plt.plot(x8, label='80%, max cases: ' + '%i' %np.max(x8))
plt.plot(x7, label='70%, max cases: ' + '%i' %np.max(x7))
plt.plot(x6, label='60%, max cases: ' + '%i' %np.max(x6))
plt.plot(x5, label='50%, max cases: ' + '%i' %np.max(x5))
plt.plot(x25, label='25%, max cases: ' + '%i' %np.max(x25))
plt.plot(x0, label='0%, max cases: ' + '%i' %np.max(x0))

plt.title('effect of compliance to self-isolation on infections. Population density: high')
plt.xlabel('simulation time steps')
plt.ylabel('number of infected')
plt.ylim(0, 2000)

plt.legend()

plt.subplot(212)
plt.plot(d99, label='99%, max fatalities: ' + '%i' %np.max(d95))
plt.plot(d95, label='95%, max fatalities: ' + '%i' %np.max(d95))
plt.plot(d9, label='90%, max fatalities: ' + '%i' %np.max(d9))
plt.plot(d8, label='80%, max fatalities: ' + '%i' %np.max(d8))
plt.plot(d7, label='70%, max fatalities: ' + '%i' %np.max(d7))
plt.plot(d6, label='60%, max fatalities: ' + '%i' %np.max(d6))
plt.plot(d5, label='50%, max fatalities: ' + '%i' %np.max(d5))
plt.plot(d25, label='25%, max fatalities: ' + '%i' %np.max(d25))
plt.plot(d0, label='0%, max fatalities: ' + '%i' %np.max(d0))

plt.title('effect of compliance to self-isolation on fatalities. Population density: high')
plt.xlabel('simulation time steps')
plt.ylabel('number of fatalities')
plt.ylim(0, 220)

plt.legend()


plt.tight_layout()
plt.show()
